#Install

In [ ]:
!pip install soccer-twos
!pip install ray==2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.1/804.1 MB 1.5 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.19.0-py3-none-any.whl size=1663114 sha256=9a5097d3a99845eacecc6f7f14a68330a41e114ae826e586b06b250bbf371260
  Stored in directory: /root/.cache/pip/wheels/11/36/28/628f4dd3779e4037a6fca1aaed76827ffa4315c3ab6bfadcf6
Successfully built gym
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.1
    Un

In [ ]:
!pip install gym==0.19.0
!pip install gym-unity==0.27.0
!pip install mlagents==0.27.0
!pip install mlagents-envs==0.27.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Imports e utils

In [ ]:
import torch
import gym
import ray
from ray import tune
from soccer_twos import EnvType
import numpy as np
import os
from ray.rllib import MultiAgentEnv
from ray.rllib.agents.ppo import ppo
from ray.tune.logger import pretty_print
import soccer_twos
import collections
import random
from collections import deque
from gym_unity.envs import ActionFlattener
import numpy as np
from tqdm import tqdm

In [ ]:
class RLLibWrapper(gym.core.Wrapper, MultiAgentEnv):
    """
    A RLLib wrapper so our env can inherit from MultiAgentEnv.
    """

    pass


def create_rllib_env(env_config: dict = {}):
    """
    Creates a RLLib environment and prepares it to be instantiated by Ray workers.
    Args:
        env_config: configuration for the environment.
            You may specify the following keys:
            - variation: one of soccer_twos.EnvType. Defaults to EnvType.multiagent_player.
            - opponent_policy: a Callable for your agent to train against. Defaults to a random policy.
    """
    if hasattr(env_config, "worker_index"):
        env_config["worker_id"] = (
            env_config.worker_index * env_config.get("num_envs_per_worker", 1)
            + env_config.vector_index
        )
    env = soccer_twos.make(**env_config)
    if "multiagent" in env_config and not env_config["multiagent"]:
        # is multiagent by default, is only disabled if explicitly set to False
        return env
    return RLLibWrapper(env)

In [ ]:
tune.registry.register_env("Soccer", create_rllib_env)
temp_env = create_rllib_env({"variation": EnvType.multiagent_player})
obs_space = temp_env.observation_space
act_space = temp_env.action_space
temp_env.close()

[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0


INFO:mlagents_envs.environment:Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0


[INFO] Connected new brain: SoccerTwos?team=1


INFO:mlagents_envs.environment:Connected new brain: SoccerTwos?team=1


[INFO] Connected new brain: SoccerTwos?team=0


INFO:mlagents_envs.environment:Connected new brain: SoccerTwos?team=0


In [ ]:
def agent_policy(id):
  if id == 0 or id == 1:
    return f"policy_0"
  else:
    return f"policy_1"

# Treinamento

In [ ]:
environment_id = "Soccer"

config = {
        "num_gpus": 1,
        "num_workers":2,
        "num_rollout_workers":1,
        "num_envs_per_worker" : 1,
        "rollout_fragment_length" : 1,
        "framework": "torch",
        "ignore_worker_failures": True,
        "train_batch_size": 256,
        "lr": 1e-3,
        "lambda": .95,
        "gamma": .99,
        "entropy_coeff": 0.01,
        "kl_coeff": 1.0,
        "clip_param": 0.2,
        "num_sgd_iter": 10,
        "vf_loss_coeff": 1e-4,
        "vf_clip_param": 100000.0,
        "multiagent": {
            "policies": {
                "policy_0": (None, obs_space, act_space, {}),
                "policy_1": (None, obs_space, act_space, {})
            },
            "policy_mapping_fn": agent_policy,
        },
        "env": "Soccer",
        "env_config": {
            "num_envs_per_worker": 1,
            "variation": EnvType.multiagent_player,
        },
        "model": {
        "fcnet_hiddens": [512, 256],
        "fcnet_activation": "tanh",
      },
    }
ppo_config = ppo.DEFAULT_CONFIG.copy()
ppo_config.update(config)

In [ ]:
import pickle


with open('/content/results2/ppo/algorithm_state.pkl', "rb") as f:
          checkpoint = pickle.load(f)

In [ ]:
trainer = ppo.PPOTrainer(config=checkpoint['config'])

(pid=40946) 2023-02-28 02:51:22,956	WARNING compression.py:16 -- lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
(RolloutWorker pid=40946) INFO:mlagents_envs.environment:Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
(RolloutWorker pid=40946) INFO:mlagents_envs.environment:Connected new brain: SoccerTwos?team=1
(RolloutWorker pid=40946) INFO:mlagents_envs.environment:Connected new brain: SoccerTwos?team=0
(RolloutWorker pid=40946) 2023-02-28 02:51:24,398	WARNING env.py:247 -- Your MultiAgentEnv <RLLibWrapper<EnvChannelWrapper<MultiAgentUnityWrapper instance>>> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__` from within your MutiAgentEnv's constructor. This will raise an error in the future.


(RolloutWorker pid=40946) [INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
(RolloutWorker pid=40946) [INFO] Connected new brain: SoccerTwos?team=1
(RolloutWorker pid=40946) [INFO] Connected new brain: SoccerTwos?team=0


2023-02-28 02:51:24,617	WARNING util.py:66 -- Install gputil for GPU system monitoring.


In [ ]:
ray.shutdown()

In [ ]:
CHECKPOINT_PATH = './results/ppo'

In [ ]:
r = []
for i in range(int(1e3)): # train iter
        r.append(trainer.train())
        print(pretty_print(r[i]))
        
        if i % 100 == 0:
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)
            trainer.save_checkpoint(CHECKPOINT_PATH)

(RolloutWorker pid=40946) 2023-02-28 02:51:41,732	WARNING deprecation.py:47 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!


A saída de streaming foi truncada nas últimas 5000 linhas.
      0.0, 0.0, 0.6908000111579895, 0.6908000111579895, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
      0.0, 0.0, 0.38179999589920044, 0.38179999589920044, -1.0, -1.0, -1.0, -1.0,
      0.0, 0.0, 0.8253999948501587, 0.8253999948501587, 0.0, 0.0, -1.0, -1.0, -1.0,
      -1.0, 0.0, 0.0, 0.9330000281333923, 0.9330000281333923, 0.0, 0.0, 0.8931999802589417,
      0.8931999802589417, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
      0.0, 0.0, -1.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
      0.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
      0.19820000231266022, 0.19820000231266022, 0.0, 0.0, 0.6395999789237976, 0.6395999789237976,
      0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0,
      -1.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0,
      -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.10140000283718109, 0.1014

In [ ]:
!zip -r '/content/drive/MyDrive/results_soccer5.zip' '/content/results/'

  adding: content/results/ (stored 0%)
  adding: content/results/ppo/ (stored 0%)
  adding: content/results/ppo/rllib_checkpoint.json (deflated 19%)
  adding: content/results/ppo/policies/ (stored 0%)
  adding: content/results/ppo/policies/policy_1/ (stored 0%)
  adding: content/results/ppo/policies/policy_1/rllib_checkpoint.json (deflated 19%)
  adding: content/results/ppo/policies/policy_1/policy_state.pkl (deflated 7%)
  adding: content/results/ppo/policies/policy_0/ (stored 0%)
  adding: content/results/ppo/policies/policy_0/rllib_checkpoint.json (deflated 19%)
  adding: content/results/ppo/policies/policy_0/policy_state.pkl (deflated 7%)
  adding: content/results/ppo/algorithm_state.pkl (deflated 69%)
